## 파이썬에서 비동기 프로그래밍 시작하기
<b> 참고자료 </b> 
https://sjquant.tistory.com/14?category=770799

### 기초
파이썬에서 비동기를 사용하기 위해서는 asyncio 모듈을 이용합니다. 함수앞에 async를 붙이면 코루틴을 만들 수 있습니다. 또한, 병목이 발생해서 다른 작업으로 통제권을 넘겨줄 필요가 있는 부분에서는 await를 써줍니다. 이때, await뒤에 오는 함수도 코루틴으로 작성되어 있어야 합니다. asyncio.sleep함수는 코루틴으로 구현되어 있기 때문에 비동기로 동작합니다. 따라서 time.sleep는 사용할 수 없습니다. (코루틴이 아닌 함수도 비동기에서 사용하는 법은 뒤에서 다룹니다.)

코루틴으로 태스크를 만들었다면, asyncio.get_event_loop함수를 활용해 이벤트 루프를 정의하고 run_until_complete으로 실행시킬 수 있습니다.

In [1]:
import asyncio
import time

import nest_asyncio
nest_asyncio.apply() # This module patches asyncio to allow nested (due to jupyter notebook)

async def sleep():
    await asyncio.sleep(5)
    
start = time.time()
loop = asyncio.get_event_loop()  # 이벤트 루프 정의
loop.run_until_complete(sleep()) # 이벤트 루프 실행
end = time.time()

print(str(end-start)+'s')

5.006123304367065s


### 비동기적 처리
비동기로 두 개 이상의 작업(코루틴)을 돌릴 때에는 asyncio.gather 함수를 이용합니다. 이때, 각 태스크들은 unpacked 형태로 넣어주어야 합니다. 즉, asyncio.gather(coroutine_1(), coroutine_2())처럼 넣어주거나 asyncio.gather(*[coroutine_1(), coroutine_2()])처럼 넣어주어야 합니다.



In [2]:
import asyncio
import time

import nest_asyncio
nest_asyncio.apply() # This module patches asyncio to allow nested (due to jupyter notebook)

async def coroutine_1():
    print("start coroutine 1")
    print("stop coroutine 1 for 5 sec")
    await asyncio.sleep(5)
    print("restart corountine 1")
    
async def coroutine_2():
    print("start coroutine 2")
    print("stop coroutine 2 for 5 sec")
    await asyncio.sleep(5)
    print("restart corountine 2")
    
if __name__ == '__main__':
    loop = asyncio.get_event_loop()

    start = time.time()
    # 코루틴이 복수 일 경우에는 asyncio.gather()를 먼저 이용하며, 순서대로 스케줄링 된다.
    loop.run_until_complete(asyncio.gather(coroutine_1(), coroutine_2()))
    end = time.time()

    print('time taken: {}'.format(end-start))

start coroutine 2
stop coroutine 2 for 5 sec
start coroutine 1
stop coroutine 1 for 5 sec
restart corountine 2
restart corountine 1
time taken: 5.006210088729858


### 코루틴으로 작성되지 않은 함수 비동기적으로 이용하기
위에서 await뒤에 오는 함수 역시 코루틴으로 작성되어 있어야 비동기적인 작업을 할 수 있다고 했습니다. 파이썬의 대부분의 라이브러리들은 비동기를 고려하지 않고 만들어졌기 때문에 비동기로 이용할 수 없습니다. 하지만, 이벤트 루프의 run_in_executor함수를 이용하면 가능합니다.

asyncio.get_event_loop()를 활용해서 현재 이벤트 루프를 loop라는 이름으로 받아오고, loop.run_in_executor를 사용하면 됩니다. 이 함수의 첫 번째 인자로는 concurrent.futures.Executor의 객체가 들어가고(None을 써주시면 asyncio의 내장 executor가 들어갑니다), 두 번째 인자로는 사용하고자 하는 함수, 그 이후의 인자(*args) 에는 사용하고자 하는 함수의 인자들을 써주면 됩니다.



In [3]:
import asyncio
import time

import nest_asyncio
nest_asyncio.apply() # This module patches asyncio to allow nested (due to jupyter notebook)

async def coroutine_1():
    print("start coroutine 1")
    print("stop coroutine 1 for 5 sec")
    
    loop = asyncio.get_event_loop()
    await loop.run_in_executor(None, time.sleep, 5)
    
    print("restart corountine 1")
    
async def coroutine_2():
    print("start coroutine 2")
    print("stop coroutine 2 for 4 sec")
    
    loop = asyncio.get_event_loop()
    await loop.run_in_executor(None, time.sleep, 4)

    print("restart corountine 2")
    
if __name__ == '__main__':
    loop = asyncio.get_event_loop()
    
    start = time.time()
    loop.run_until_complete(asyncio.gather(coroutine_1(), coroutine_2()))
    end = time.time()
    
    print('time taken: {}'.format(end-start))

start coroutine 2
stop coroutine 2 for 4 sec
start coroutine 1
stop coroutine 1 for 5 sec
restart corountine 2
restart corountine 1
time taken: 5.007379055023193


### run_in_executor 과 asyncio.sleep를 이용하여 성능 비교하기
asyncio.sleep을 사용한 것과 거의 유사한 결과를 볼 수 있습니다. 원리는 무엇일까요? 사실 이것은 비동기적 처리처럼 보이지만 실제로는 쓰레딩을 이용한 것이라고 할 수 있습니다. 첫번째 글에서 언급했던 멀티쓰레드기억나시나요? 비동기적 처리보다는 비효율적이었지만 작업이 완료되길 기다리고 다른 작업을 시작하는 것보다는 빠르게 작업을 처리할 수 있었습니다.

하지만, 쓰레딩을 이용했을 때는 비용도 만만치 않았습니다. 파이썬에서는 GIL 때문에 쓰레드들이 동시에 작업이 불가능하기 때문에, 다른 쓰레드를 호출하는데 걸리는 시간을 낭비하게 됩니다. (컨텍스트 스위칭의 비용)

In [4]:
import asyncio
import time
from concurrent.futures import ThreadPoolExecutor

async def sleep(executor=None):
    loop = asyncio.get_event_loop()
    await loop.run_in_executor(executor, time.sleep, 1)

async def main():

    # max_workers에 따라서 실행시간이 달라지는 것을 확인할 수 있다.
    # (하지만 workers가 많아질수록 컨텍스트 스위칭 비용도 커진다.)
    # None으로 하는 경우는 디폴트로 설정한 workers수가 작아서 인지 훨씬 더 오래걸린다.

    executor = ThreadPoolExecutor(max_workers=10000)

    # asyncio.ensure_future함수는 태스크를 현재 실행하지 않고,
    # 이벤트 루프가 실행될 때 실행할 것을 보증해주는 함수
    futures = [asyncio.ensure_future(sleep(executor)) for i in range(10000)]
    await asyncio.gather(*futures)

if __name__ == "__main__":
    start = time.time()
    # python 3.7부터는 이벤트 루프를 따로 명시적으로 지정하지 않고, asyncio.run으로 돌릴 수 있다.
    asyncio.run(main())
    end = time.time()
    print('{}'.format(end-start))

5.235754489898682


In [5]:
import asyncio
import time

async def sleep():
    await asyncio.sleep(1)

async def main():
    # asyncio.sleep은 아무리 많아져도 비동기적으로 잘 돌아간다.
    futures = [asyncio.ensure_future(sleep()) for i in range(10000)]
    await asyncio.gather(*futures)

if __name__ == "__main__":
    start = time.time()
    asyncio.run(main())
    end = time.time()
    print('{}'.format(end-start))

1.5505709648132324


### 비동기 프로그래밍 사례
#### 동기적 방식
비동기를 사용하면 네트워크 IO의 지연 때문에 낭비되는 시간을 줄일 수 있습니다. 온라인 사전사이트(네이버)에서 단어들의 의미를 크롤링하는 코드를 작성한다고 가정해봅시다. 동기적인 방식을 사용한다면 아래와 같이 코드를 작성할 수 있습니다.

In [6]:
import requests
import time
from bs4 import BeautifulSoup

def get_text_from_url(url):
    print("Send request to {url}".format(url=url))
    res = requests.get(url, headers={'uesr-agent': 'Mozilla/5.0'})
    print("Get request from {url}".format(url=url))
    text = BeautifulSoup(res.text, 'html.parser').text
    return text

if __name__ == '__main__':
    start = time.time()
    
    base_url = 'https://endic.naver.com/search.nhn?sLn=kr&dicQuery={keyword}&query={keyword}&target=endic&ie=utf8&query_utf=&isOnlyViewEE=N'
    keywords = ['apple', 'banana', 'call', 'feel',
                'hello', 'bye', 'like', 'love', 'environmental',
                'buzz', 'ambition', 'determine']

    urls = [base_url.format(keyword=keyword) for keyword in keywords]
    for url in urls:
        text = get_text_from_url(url)
        print("=" * 150)            
        #print(text[8100:9000].strip())
        
    end = time.time()
    print("time taken: {}".format(end-start))

Send request to https://endic.naver.com/search.nhn?sLn=kr&dicQuery=apple&query=apple&target=endic&ie=utf8&query_utf=&isOnlyViewEE=N
Get request from https://endic.naver.com/search.nhn?sLn=kr&dicQuery=apple&query=apple&target=endic&ie=utf8&query_utf=&isOnlyViewEE=N
Send request to https://endic.naver.com/search.nhn?sLn=kr&dicQuery=banana&query=banana&target=endic&ie=utf8&query_utf=&isOnlyViewEE=N
Get request from https://endic.naver.com/search.nhn?sLn=kr&dicQuery=banana&query=banana&target=endic&ie=utf8&query_utf=&isOnlyViewEE=N
Send request to https://endic.naver.com/search.nhn?sLn=kr&dicQuery=call&query=call&target=endic&ie=utf8&query_utf=&isOnlyViewEE=N
Get request from https://endic.naver.com/search.nhn?sLn=kr&dicQuery=call&query=call&target=endic&ie=utf8&query_utf=&isOnlyViewEE=N
Send request to https://endic.naver.com/search.nhn?sLn=kr&dicQuery=feel&query=feel&target=endic&ie=utf8&query_utf=&isOnlyViewEE=N
Get request from https://endic.naver.com/search.nhn?sLn=kr&dicQuery=feel&qu

#### 비동기적 방식 (Asyncio + requests)
이번에는 비동기적으로 코드를 작성해봅시다. requests는 비동기적으로 작성되지 않았기 때문에 loop.run_in_executor를 통해 쓰레드를 만드는 방식을 사용합니다.

In [7]:
import requests
import time
import asyncio
from functools import partial
from bs4 import BeautifulSoup


async def get_text_from_url(url):  # 코루틴 정의
    print("Send request to {url}".format(url=url))
    loop = asyncio.get_event_loop()

    # loop.run_in_executor는 kwargs(keyword arguments)를 사용할 수 없기 때문에 functools.partial을 활용
    request = partial(requests.get, url, headers={'user-agent': 'Mozilla/5.0'})
    # ascyncio의 디폴트 쓰레드풀을 사용할 경우 첫번째 인자로 None
    # 직접 쓰레드풀을 만들 경우 concurrent.futures.threadpoolexecutor 사용
    res = await loop.run_in_executor(None, request)
    print("Get request from {url}".format(url=url))
    text = BeautifulSoup(res.text, 'html.parser').text
    print("=" * 150)    
    #print(text[8100:9000].strip())


async def main():
    base_url = 'https://endic.naver.com/search.nhn?sLn=kr&dicQuery={keyword}&query={keyword}&target=endic&ie=utf8&query_utf=&isOnlyViewEE=N'
    keywords = ['apple', 'banana', 'call', 'feel',
                'hello', 'bye', 'like', 'love', 'environmental',
                'buzz', 'ambition', 'determine']

    # 아직 실행된 것이 아니라, 실행할 것을 계획하는 단계
    futures = [asyncio.ensure_future(get_text_from_url(
        base_url.format(keyword=keyword))) for keyword in keywords]

    await asyncio.gather(*futures)

if __name__ == "__main__":
    start = time.time()
    loop = asyncio.get_event_loop()
    loop.run_until_complete(main())
    end = time.time()
    print("time taken: {}".format(end-start))

Send request to https://endic.naver.com/search.nhn?sLn=kr&dicQuery=apple&query=apple&target=endic&ie=utf8&query_utf=&isOnlyViewEE=N
Send request to https://endic.naver.com/search.nhn?sLn=kr&dicQuery=banana&query=banana&target=endic&ie=utf8&query_utf=&isOnlyViewEE=N
Send request to https://endic.naver.com/search.nhn?sLn=kr&dicQuery=call&query=call&target=endic&ie=utf8&query_utf=&isOnlyViewEE=N
Send request to https://endic.naver.com/search.nhn?sLn=kr&dicQuery=feel&query=feel&target=endic&ie=utf8&query_utf=&isOnlyViewEE=N
Send request to https://endic.naver.com/search.nhn?sLn=kr&dicQuery=hello&query=hello&target=endic&ie=utf8&query_utf=&isOnlyViewEE=N
Send request to https://endic.naver.com/search.nhn?sLn=kr&dicQuery=bye&query=bye&target=endic&ie=utf8&query_utf=&isOnlyViewEE=N
Send request to https://endic.naver.com/search.nhn?sLn=kr&dicQuery=like&query=like&target=endic&ie=utf8&query_utf=&isOnlyViewEE=N
Send request to https://endic.naver.com/search.nhn?sLn=kr&dicQuery=love&query=love&t

#### 비동기적 방식 (asyncio + aiohttp)
하지만, requests모듈은 코루틴으로 만들어진 모듈이 아니기 때문에 위의 코드는 내부적으로 쓰레드를 만들어 동작합니다. 따라서, 요청의 수가 많아질수록 컨텍스트 스위칭의 비용이 발생합니다. 비동기 HTTP통신 라이브러리인 aiohttp를 이용하면 코루틴을 이용한 비동기 방식을 이용할 수 있습니다.

In [8]:
import time
import asyncio
# aiohttp 설치 필요
import aiohttp
from bs4 import BeautifulSoup


async def get_text_from_url(url):  # 코루틴 정의
    print("Send request to {url}".format(url=url))

    async with aiohttp.ClientSession() as sess:
        async with sess.get(url, headers={'user-agent': 'Mozilla/5.0'}) as res:
            text = await res.text()

    print("Get request to {url}".format(url=url))
    text = BeautifulSoup(text, 'html.parser').text
    print("=" * 150)
    #print(text[8000:8300].strip())


async def main():
    base_url = 'https://endic.naver.com/search.nhn?sLn=kr&dicQuery={keyword}&query={keyword}&target=endic&ie=utf8&query_utf=&isOnlyViewEE=N'
    keywords = ['apple', 'banana', 'call', 'feel',
                'hello', 'bye', 'like', 'love', 'environmental',
                'buzz', 'ambition', 'determine']

    # 아직 실행된 것이 아니라, 실행할 것을 계획하는 단계
    futures = [asyncio.ensure_future(get_text_from_url(
        base_url.format(keyword=keyword))) for keyword in keywords]

    await asyncio.gather(*futures)

if __name__ == "__main__":

    start = time.time()
    loop = asyncio.get_event_loop()
    loop.run_until_complete(main())
    end = time.time()
    print("time taken: {}".format(end-start))

Send request to https://endic.naver.com/search.nhn?sLn=kr&dicQuery=apple&query=apple&target=endic&ie=utf8&query_utf=&isOnlyViewEE=N
Send request to https://endic.naver.com/search.nhn?sLn=kr&dicQuery=banana&query=banana&target=endic&ie=utf8&query_utf=&isOnlyViewEE=N
Send request to https://endic.naver.com/search.nhn?sLn=kr&dicQuery=call&query=call&target=endic&ie=utf8&query_utf=&isOnlyViewEE=N
Send request to https://endic.naver.com/search.nhn?sLn=kr&dicQuery=feel&query=feel&target=endic&ie=utf8&query_utf=&isOnlyViewEE=N
Send request to https://endic.naver.com/search.nhn?sLn=kr&dicQuery=hello&query=hello&target=endic&ie=utf8&query_utf=&isOnlyViewEE=N
Send request to https://endic.naver.com/search.nhn?sLn=kr&dicQuery=bye&query=bye&target=endic&ie=utf8&query_utf=&isOnlyViewEE=N
Send request to https://endic.naver.com/search.nhn?sLn=kr&dicQuery=like&query=like&target=endic&ie=utf8&query_utf=&isOnlyViewEE=N
Send request to https://endic.naver.com/search.nhn?sLn=kr&dicQuery=love&query=love&t